In [ ]:
# this cell is required to run these notebooks on Binder
# if running on Binder, remember to **WAIT 5 SECONDS** after
# running the '%gui qt' cell below. If you get an error,
# click on "Kernel -> Restart" and try again. Make sure also
# that you have a desktop tab open.
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    os.environ['DISPLAY'] = ':1.0'

In [1]:
##############################################################################################################
##############################################################################################################

# CHANGE HERE SAMPLE :
sample = "ROI-09"

##############################################################################################################
##############################################################################################################

In [2]:
import os
import pandas as pd
import numpy as np
from tifffile import imread
import random

%gui qt

In [3]:
import napari
from napari.utils import nbscreenshot

# Create an empty viewer
viewer = napari.Viewer()

In [4]:
# One time only select random samples from each category
print(["1","7","13","18","28"][random.randint(0,4)]) # HD
print(["2","8","14","19","24"][random.randint(0,4)]) # LP
print(["3","9","15","20","25"][random.randint(0,4)]) # AD
print(["4","10","16","21","26"][random.randint(0,4)]) # PS
print(["5","11","17","22"][random.randint(0,3)]) # SS
print(["6","12","23","27"][random.randint(0,3)]) # MF

In [5]:
HD = ["ROI-13", "ROI-07"]
LP = ["ROI-08", "ROI-14"]
AD = ["ROI-25", "ROI-20"]
PS = ["ROI-4", "ROI-10"]
SS = ["ROI-22", "ROI-05"]
MF = ["ROI-12", "ROI-06"]

samples = [HD, LP, AD, PS, SS, MF]
samples = [item for sublist in samples for item in sublist]
samples

In [6]:
["TL","BL","TR","BR"][random.randint(0,3)]

In [7]:
print(["Christoph","YunTsan","YiChien","Ionoss","Pacome"][random.randint(0,4)])
print(samples[random.randint(0,len(samples)-1)])

In [8]:
Christoph = ["ROI-22", "ROI-07", "ROI-23-BL", "ROI-17-TL"]
YunTsan = ["ROI-13", "ROI-25", "ROI-23-BL"]
YiChien = ["ROI-04", "ROI-05","ROI-17-TL"]
Ionoss = ["ROI-20", "ROI-14", "ROI-17-TL"]
Pacome = ["ROI-08", "ROI-12","ROI-23-BL", "ROI-17-TL"]


In [9]:
# Change below
output_dir = "/media/localadmin/T7/CHUV/Documents/Muliplex_Imaging_Pipeline/output/manual_annotation"
cell_marker_file = "/media/localadmin/T7/CHUV/Documents/Muliplex_Imaging_Pipeline/annotation/cell_markers.csv"
tiff_dir = "/media/localadmin/T7/CHUV/Documents/Muliplex_Imaging_Pipeline/output/input/"
segmentation_dir  = "/media/localadmin/T7/CHUV/Documents/Muliplex_Imaging_Pipeline/output/segmentation/"
corner = ["TL","BL","TR","BR"][random.randint(0,3)]

In [10]:
save_dir = os.path.join(output_dir, sample + "-" + corner)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    

In [11]:
cell_type = pd.read_csv(os.path.join(cell_marker_file))

cell_types_unique = cell_type.cell_type[np.sort(np.unique(cell_type.cell_type, return_index=True)[1])]
cell_types_unique = np.flip(cell_types_unique)

marker_unique = cell_type.marker[np.sort(np.unique(cell_type.marker, return_index=True)[1])]
marker_unique = np.flip(marker_unique)


In [12]:
#DAPI = imread(os.path.join(os.path.join(tiff_dir, sample, "DAPI.tiff")))
#viewer.add_image(DAPI, name="DAPI", colormap="gray",contrast_limits=[7500, 50000]);


In [13]:
for i in cell_types_unique:
    df = cell_type[cell_type.cell_type == i]
    chain = i
    for marker in df.marker:
        if marker not in viewer.layers:
            color = cell_type.color_map[cell_type.marker == marker].to_list()[0]
            min_contrast = cell_type.default_min[cell_type.marker == marker].to_list()[0]
            max_contrast = cell_type.default_max[cell_type.marker == marker].to_list()[0]
            visible = cell_type.visible[cell_type.marker == marker].to_list()[0]
            blending = cell_type.blending[cell_type.marker == marker].to_list()[0]
            image = imread(os.path.join(os.path.join(tiff_dir, sample, marker + ".tiff")))
            viewer.add_image(image, name=marker, colormap=color, contrast_limits=[int(min_contrast), int(max_contrast)],
                            blending = blending, visible = visible);


In [14]:
chains = []
for i in cell_types_unique:
    
    df = cell_type[cell_type.cell_type == i]
    chain = i
    for marker in df.marker:
        if np.asarray(df.positive[df.marker == marker])[0] == True:
            chain = chain + str("_") + marker + "+"
        else:
            chain = chain + str("_") + marker + "-"
    print(chain)
    chains = chains + [chain]
    viewer.add_points(name=chain, face_color = df.cell_type_color.to_list()[0],
                      edge_color = "#d4c51bff", edge_width=0.1, opacity=1, size = 30);
    viewer.layers[chain].mode = "add"

In [15]:
if corner == "TL":
    polygon = [[0, 0],[round(image.shape[0] /2),round(image.shape[1] /2)]]
if corner == "TR":
    polygon = [[0, round(image.shape[1] /2)],[round(image.shape[0] /2),image.shape[1]]]
if corner == "BL":
    polygon = [[round(image.shape[0] /2), 0],[image.shape[0],round(image.shape[1] /2)]]
if corner == "BR":
    polygon = [[round(image.shape[0] /2), round(image.shape[1] /2)], [image.shape[0], image.shape[1]]]

viewer.add_shapes(polygon, name="ROI", face_color = "#77777700", edge_color = "red", edge_width=20)

In [16]:
segmentation = imread(os.path.join(os.path.join(segmentation_dir, sample + "_whole_cell_segmentation_borders.tiff")))
viewer.add_labels(segmentation, name="cell_types");

In [17]:
@viewer.bind_key('Shift-S', overwrite = "True")
def save_all(viewer):
    for i in chains:
        viewer.layers[i].save(os.path.join( save_dir, i + '.csv'));